# Constituency Parsing
Syntactic parsing is the task of recognizing a sentence and assigning a syntactic structure to it.  
  
Ambiguity is the most serious problem faced by syntactic parsers. Structural ambiguity occurs when the grammar can assign more than one parse to a sentence. Two common kinds of it are attachment ambiguity and coordination ambiguity.  
  
A sentence has an attachment ambiguity if a particular constituent can be attached to the parse tree at more than one place.  
  
In coordination ambiguity different sets of phrases can be conjoined by a conjunction like and.

# Chomsky Normal Form
Cocke-Kasami-Younger (CKY) algorithm is the most widely used dynamic-programming based approach to parsing, and it requires that grammars used with it must be in Chomsky Normal Form (CNF). In CNF, the right-hand side of each rule must expand either to two nonterminals or to a single terminal.

## Converting CFG to CNF
There are three situations we need to solve:
* rules that mix terminals with non-terminals on the right-hand side
* rules that have a single non-terminal on the right-hand side
* rules in which the length of the right-hand side is greater than 2

For mix terminals and non-terminals, we can simply introduce a new dummy non-terminal that covers only the original terminal. e.g. INF-VP → to VP can be replaced by INF-VP → TO VP and TO → to

Rules with right-hand sides longer than 2 are normalized through the introduction of new non-terminals that spread the longer sequences over several new rules.

Rules with a single non-terminal on the right are called unit productions. We can eliminate unit productions by rewriting the right-hand side of the original rules with the right-hand side of all the non-unit production rules that they ultimately lead to.

In [3]:
import nltk
from nltk.grammar import CFG, Nonterminal, Production

cfg = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N N N | NP PP | Det Adj N 
VP -> V | V NP | VP PP
Det -> 'a' | 'the'
Adj -> 'quick' | 'brown' 
N -> 'dog' | 'cat' | 'mat'
V -> 'chased' | 'sat'
P -> 'on' | 'in'
NP -> 'the' Nom
""")

c = 0 # count of intermediate rule X_c
to_convert = [] # for rules to be converted except mixed cases
mixed = [] # for mix terminals and non-terminals
result = []

for p in cfg.productions():
    if p.is_lexical() and len(p.rhs()) > 1:
        mixed.append(p)
    else:
        to_convert.append(p)

for p in mixed:
    l, r = p.lhs(), p.rhs()
    new_r = []
    for t in r:
        if type(t) == str: 
            new_r.append(Nonterminal('_X{}_'.format(c)))
            result.append(Production(Nonterminal('_X{}_'.format(c)), (t,)))
            c += 1
        else:
            new_r.append(t)
    if len(new_r) == 2:
        result.append(Production(l, new_r))
    else:
        to_convert.append(Production(l, new_r))

for rule in to_convert:
    if len(rule.rhs()) > 2:
        l = rule.lhs()
        for k in range(0, len(rule.rhs()) - 2):
            r = rule.rhs()[k]
            inter = Nonterminal('_X{}_'.format(c))
            c += 1
            new_rule = Production(l, (r, inter))
            l = inter
            result.append(new_rule)
        last = Production(l, rule.rhs()[-2:])
        result.append(last)
    else:
        result.append(rule)
cnf = CFG(cfg.start(), result)

result = []
unitary = [] # For rules with a single non-terminal

for rule in cnf.productions():
    if len(rule) == 1 and rule.is_nonlexical():
        unitary.append(rule)
    else:
        result.append(rule)

while unitary:
    rule = unitary.pop(0)
    for item in cnf.productions(lhs=rule.rhs()[0]):
        new_rule = Production(rule.lhs(), item.rhs())
        if len(new_rule) != 1 or new_rule.is_lexical():
            result.append(new_rule)
        else:
            unitary.append(new_rule)

cnf = CFG(cnf.start(), result)

In [5]:
print(cnf)

Grammar with 25 productions (start state = S)
    _X0_ -> 'the'
    NP -> _X0_ Nom
    S -> NP VP
    PP -> P NP
    NP -> Det _X1_
    _X1_ -> N _X2_
    _X2_ -> N N
    NP -> NP PP
    NP -> Det _X3_
    _X3_ -> Adj N
    VP -> V NP
    VP -> VP PP
    Det -> 'a'
    Det -> 'the'
    Adj -> 'quick'
    Adj -> 'brown'
    N -> 'dog'
    N -> 'cat'
    N -> 'mat'
    V -> 'chased'
    V -> 'sat'
    P -> 'on'
    P -> 'in'
    VP -> 'chased'
    VP -> 'sat'


# CKY Parsing
Cocke-Kasami-Younger parsing algorithm is an efficient bottom-up parsing algorithm based on tabulating substring parses to avoid repeated work.

Approach:
* use a CNF grammar
* build a matrix to store subtrees
* incrementally build parse spanning whole input string

The complexity of CKY Parsing is $O(n^3)$

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tree import Tree

def parse(sent, grammar):
    '''
    sent: list of words
    grammar: NLTK grammar
    '''
    table = [[None]*len(sent) for _ in range(len(sent))]
    for j in range(len(sent)):
        table[j][j] = [(rule.lhs(), Tree(str(rule.lhs()), rule.rhs())) for rule in grammar.productions(rhs=sent[j])]
        for i in range(j)[::-1]:
            for k in range(i, j):
                l1, l2 = table[i][k], table[k+1][j]
                l3 = table[i][j] if table[i][j] else []
                if l1 and l2:
                    for i1 in l1:
                        for i2 in l2:
                            s1, tree1 = i1
                            s2, tree2 = i2
                            possible_rule = grammar.productions(rhs=s1)
                            for r in possible_rule:
                                if r.rhs() == (s1, s2):
                                    l3 += [(r.lhs(), Tree(str(r.lhs()), [tree1, tree2]))]
                    table[i][j] = l3
    return table
    
def print_tree(table):
    total = len(table[0][-1])
    for tree in table[0][-1]:
        if tree[0] == grammar.start():
            print(tree[1])

def parse_sentence(s, grammar):
    print(s)
    print_tree(parse(word_tokenize(s), grammar))

In [12]:
grammar_filename = 'data/grammar_cnf.cfg'
grammar = nltk.data.load(grammar_filename)

text = 'Scientists have discovered that malaria invades the whole body.'

parse_sentence(text,grammar)

Scientists have discovered that malaria invades the whole body.
(TOP
  (NP Scientists)
  (VP
    (AUX have)
    (VP
      (VBN discovered)
      (SBAR
        (_X_2 that)
        (S
          (NP malaria)
          (_X_5
            (VP
              (VBZ invades)
              (NP (Det the) (Nom (ADJP whole) (Nom body))))
            (PUNC .)))))))
(TOP
  (NP Scientists)
  (_X_6
    (VP
      (AUX have)
      (VP
        (VBN discovered)
        (SBAR
          (_X_2 that)
          (S
            (NP malaria)
            (VP
              (VBZ invades)
              (NP (Det the) (Nom (ADJP whole) (Nom body))))))))
    (PUNC .)))
